In [ ]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import time

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

def generate_report(dd):
    
    import numpy as np
    import os
    import glob
    import datetime
    import numpy as np
    import pandas as pd
    import time
    import pickle

    
    print('start')
    readPath = '/mnt/orderLog/data/***'
    dataPathLs = np.array(glob.glob(readPath))
    path = np.sort(dataPathLs)[-11:]
    cur = np.max([int(os.path.basename(i).split('.')[0]) for i in path])
    assert(cur == int(dd))


    body = "<html><body><div>" + 'Hi all,<p>The following is daily report based on ' + str(cur) + ' data.</p>'
    body += '<p><b>I. Assertions</p></b>'
    count = 0

    # load data
    print('-----------------------------------------------------------------------------------------------')
    print('load data')
    rawOrderLog = []
    for thisPath in path:
        print(thisPath)
        data = pickle.load(open(thisPath, 'rb'))
        data = data.rename(columns={'mdClockAtArrival': 'caamd'})
        rawOrderLog += [data]
    rawOrderLog = pd.concat(rawOrderLog, sort=False)

    for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
    rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

    if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
        print('There are accounts with duplicated ticks:')
        print(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)]\
    .groupby(['date', 'colo', 'accCode'])['ars'].size())
        rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep='first')
    
    
    print('There are ticks with orderDirection 0')
    print(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
                'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])
    try:
        num1 = rawOrderLog[(rawOrderLog['orderDirection'] == 0) & (rawOrderLog['date'] == cur)].shape[0]
        assert(num1 < 30)
    except:
        count += 1
        body += str(count) + '. There are in total ' + str(num1) + ' orders with orderDirection 0.<div>'


    assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    try:
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    except:
        print('There are orders with all things same except sdd')
        print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)])
        assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
    try:
        assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
    except:
        print('There are orders with same internalId but different orderId other than accCode 8856 case')
        print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

    r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
    r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize']).grouper.group_info[0]
    r1 = r1.sort_values(by=['test', 'clockAtArrival'])
    r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
    r1['vai'] = r1.groupby('test')['vai'].ffill()
    r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    r2 = r2.sort_values(by=['test', 'clockAtArrival'])
    r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
    r2['vai'] = r2.groupby('test')['vai'].ffill()
    try:
        assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(pd.merge(r1, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    
    try:
        assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print('There are orders out of 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    orderLog = pd.concat([r1, r2])
    del r1
    del r2    

    orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
    orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog['accCode'] // 100)
    orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
    orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
    orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
    orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
    orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
    orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
        orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
    orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
    orderLog['sdd'] = orderLog.groupby('order')['sdd'].transform('first')
    orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')


    ### Assertion 1:  make sure same direction in same date, secid, vai
    print('=======================================================================================')
    print('1. same date, secid, vai: same direction')
    taking = orderLog[orderLog['ars'].isin([1, 7])]
    making = orderLog[orderLog['ars'].isin([2, 3])]
    else_orders = orderLog[~(orderLog['ars'].isin([1, 2, 3, 7]))]
    if else_orders.shape[0] != 0:
        print('orders with abnormal ars values')
        print(else_orders[(else_orders['updateType'] == 0) & (else_orders['date'] == cur)].groupby('ars')['date'].size().sort_values(ascending=False))
        print(else_orders[(else_orders['updateType'] == 0) & (else_orders['date'] == cur)].groupby('accCode')['date'].size().sort_values(ascending=False))
        ars_list = else_orders[(else_orders['updateType'] == 0) & (else_orders['date'] == cur)].groupby('ars')['date'].size().sort_values(ascending=False).reset_index()['ars'].values
        ars_list = ', '.join([str(x) for x in ars_list])
        count += 1
        body += str(count) + '. There are abnormal ars values in data: ' + ars_list + '.<div>'
        kk = else_orders[(else_orders['updateType'] == 0) & (else_orders['date'] == cur)].groupby('accCode')['date'].size().sort_values(ascending=False)
        if kk[kk > 20].shape[0] > 0:
            count += 1
            stock_list = ', '.join([str(x) for x in kk[kk > 20].index.values])
            body += str(count) + '. These accounts have more than 20 abnormal ars orders: ' + stock_list + '.<div>'
    taking['directNum'] = taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].transform('nunique')
    if taking[(taking['directNum'] != 1)].shape[0] > 0:
        print('opposite direction for same date, same secid, same vai')
        print(taking[(taking['directNum'] != 1) & (taking['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
        try:
            num1 = taking[(taking['directNum'] != 1) & (taking['updateType'] == 0) & (taking['date'] == cur)].shape[0]
            assert(num1 < 20)
        except:
            count += 1
            body += str(count) + '. There are in total ' + str(num1) + ' orders with opposite directions under same date, secid, vai.<div>'
        try:
            num2 = list(taking[(taking['directNum'] != 1) & (taking['updateType'] == 0) & (taking['date'] == \
                cur)].groupby('accCode')['date'].size()[taking[(taking['directNum'] != 1) & \
               (taking['updateType'] == 0) & (taking['date'] == cur)].\
                                                                           groupby('accCode')['date'].size() > 10].index)
            assert(len(num2) == 0)
        except:
            count += 1
            num2 = ', '.join([str(x) for x in num2])
            body += str(count) + '. ' + num2 + ' has more than 10 orders with opposite directions under same date, secid, vai.<div>'
        taking = taking[taking['directNum'] == 1]

    assert((taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].nunique() == 1).all() == True)
    orderLog = pd.concat([taking, making]).sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)

    ## Assertion 2:  make sure each account, secid, vai only has one insertion
    print('=======================================================================================')
    print('2. same date, secid, vai, accCode: one insertion')
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['clockAtArrival'].count().reset_index()
    if a[a['clockAtArrival'] > 1].shape[0] > 0:
        print('more than one insertion at same time')
        a = a[(a['clockAtArrival'] > 1)]
        print(a)
        print(a.groupby(['date'])['accCode'].size())
        print(a.groupby(['date', 'accCode'])['sdd'].size())
        try:
            num1 = a[a['date'] == cur].shape[0]
            assert(num1 < 20)
        except:
            count += 1
            body += str(count) + '. There are in total ' + str(num1) + ' orders with more than one insertion in same order.<div>'
        try:
            num2 = list(a[a['date'] == cur].groupby(['accCode'])['date'].size()[a[a['date'] == cur].groupby(['accCode'])['date'].size() > 10].index)
            assert(len(num2) == 0)
        except:
            count += 1
            num2 = ', '.join([str(x) for x in num2])
            body += str(count) + '. ' + num2 + ' has more than 10 orders with more than one insertion in same order.<div>'        
        d_el = pd.merge(orderLog, a, on=['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(d_el))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()


    ### Assertion 3:  check IPO stocks selling status
    print('=======================================================================================')
    print('3. IPO stocks selling (ars = 301, 302)')
    if orderLog[(orderLog['ars'].isin([301, 302])) & (orderLog['date'] == cur)].shape[0] != 0:
        kk = orderLog[(orderLog['ars'].isin([301, 302])) & (orderLog['date'] == cur)]
        print(kk)
        try:
            assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
            print('we only sell, never buy')
        except:
            print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            count += 1
            num1 = kk[kk['orderDirection1'] == 1].shape[0]
            body += str(count) + '. There are ' + str(num1) + ' IPO buy side orders.<div>'
            print(kk[kk['orderDirection1'] == 1])
        kk1 = kk[kk['updateType'] == 0]
        kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
        kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
        kk1['diff'] = kk1['diff'].fillna(0)
        try:
            assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
            print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
        except:
            count += 1
            kk1 = kk1.reset_index()
            num2 = kk1[kk1['diff'] < 10e6].shape[0]
            body += str(count) + '. There are ' + str(num2) + ' over ' + str(kk1.shape[0]) + ' orders with insertion within 10 seconds for orders under same account same stock.<div>'
            print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')        
            print(kk1[kk1['diff'] < 10e6])
        kk2 = kk[(kk['updateType'] == 1)]

        try:
            assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
            print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
        except:
            count += 1
            num2 = kk2[kk2['duration'] < 3e6].shape[0]
            body += str(count) + '. There are ' + str(num2) + ' over ' + str(kk1.shape[0]) + ' orders with cancellation within 3 seconds after insertion.<div>'        
            print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk2[kk2['duration'] < 3e6])


    ### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
    print('=======================================================================================')
    print('4. updateType 7 orders')
    if orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].shape[0] != 0:
        try:
            assert(orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
        except:
            print('There are more than 20 updateType 7 orders per account')
            count += 1
            a = list(orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique()[
                orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique() >= 20
            ].index)
            a = ', '.join([str(x) for x in a])
            body += str(count) + '. ' + a + ' has more than 20 updateType 7 orders.<div>'
        try:      
            assert(orderLog[(orderLog['updateType'] == 7) & (orderLog['date'] == cur)]['order'].nunique() < 100)
        except:
            print('Ther are more than 100 updateType 7 orders in total')
            count += 1
            body += str(count) + '. There are more than 100 updateType 7 orders in total.<div>'


    ### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
    ### order being rejected by broker
    print('=======================================================================================')
    print('5. updateType 6 orders')
    k1 = orderLog[(orderLog['updateType'] == 6) & (orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique().reset_index()
    k2 = orderLog[(orderLog['date'] == cur)].groupby(['accCode'])['order'].nunique().reset_index()
    k = pd.merge(k1, k2, on=['accCode'], how='left')
    k['prob'] = k['order_x']/k['order_y']
    try:
        assert(sum(k['prob'] >= 0.05) == 0)
    except:
        print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(k[k['prob'] >= 0.05])
        a = k[k['prob'] >= 0.05]['accCode'].unique()
        a = ', '.join([str(x) for x in a])
        count += 1
        body += str(count) + '. ' + a + ' has more than 5% updateType 6 orders.<div>'

    ### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
    print('=======================================================================================')
    print('6. CYB stocks order size < 30w')
    try:
        cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0) & (orderLog['date'] == cur)]
        assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
    except:
        print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        num1 = cyb[cyb['absOrderSize'] > 300000].shape[0]
        count += 1
        body += str(count) + '. There are ' + str(num1) + ' orders with CYB absOrderSize > 30w.<div>'


    ### Assertion 7:  make sure there is no unexpected updateType 
    print('=======================================================================================')
    print('7. unexpected updateType')
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                         np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                                   (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                                   (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                         np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))
    print(checkLog[checkLog['status'] == 3].groupby('updateType')['order'].size())
    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### Assertion 8:  make sure status==0 got all traded
    print('=======================================================================================')
    print('8. status == 0: all traded')
    a = orderLog[(orderLog['status'] == 0)]
    a = a.groupby(['date', 'order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['date', 'order', 'filled', 'total']
    print('in total trade, any fill != total cases')
    print(a[a['filled'] != a['total']])
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        count += 1
        body += str(count) + '. There are ' + str(a[(a['filled'] != a['total']) & (a['date'] == cur)]['order'].nunique()) + \
        ' over ' + str(a[(a['date'] == cur)]['order'].nunique()) + ' orders status == 0 but not all traded.<div>'
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 9:  make sure status==1 got partial traded
    print('=======================================================================================')
    print('9. status == 1: partial traded')
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['date', 'order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['date', 'order', 'filled', 'total']
    print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
    print(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        count += 1
        body += str(count) + '. There are ' + str(a[((a['filled'] >= a['total']) | (a['filled'] == 0)) & (a['date'] == cur)]['order'].nunique()) + \
        ' over ' + str(a[(a['date'] == cur)]['order'].nunique()) + ' orders status == 1 but not partial traded.<div>'
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 10: make sure no cancellation within 1 sec
    print('=======================================================================================')
    print('10. no cancellation within 1 sec')
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    print('any cancellation within 1 sec')
    print(a)
    if a[a['date'] == cur].shape[0] > 0:
        count += 1
        body += str(count) + '. There are ' + str(a[(a['date'] == cur)]['order'].nunique()) + ' orders cancel within 1s.<div>'    
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


    ### Assertion 11: make sure no order has shares > 80w or notional > 800w
    print('=======================================================================================')
    print('11. Orders with size > 80w or notional > 800w')
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
    if orderLog[(orderLog['absOrderSize'] > 800000) & (orderLog['date'] == cur)].shape[0] > 0:
        count += 1
        body += str(count) + '. There are ' + str(orderLog[(orderLog['absOrderSize'] > 800000) & (orderLog['date'] == cur)]['order'].nunique()) + ' orders shares > 80w.<div>'    
    if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
        print('some order quantity are > 80w')
        print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
        print(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                             'orderNtl', 'orderDirection', 'clock', 'order']])
    if orderLog[(orderLog['orderNtl'] > 8000000) & (orderLog['date'] == cur)].shape[0] > 0:
        count += 1
        body += str(count) + '. There are ' + str(orderLog[(orderLog['orderNtl'] > 8000000) & (orderLog['date'] == cur)]['order'].nunique()) + ' orders notional > 800w.<div>'                
    if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
        print('some order ntl are > 800w')
        print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
        print(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                          'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                          "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

    removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'order', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
    orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
    orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
    orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
    orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
    orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')
    orderLog = orderLog[~orderLog['ars'].isnull()]
    # orderLog = orderLog[orderLog['ars'] % 10 == 1]


    orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    try:
        orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
    except:
        print(orderLog[orderLog['mrsb90'] == '-'])
        orderLog = orderLog[orderLog['mrsb90'] != '-']
        orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
    try:
        orderLog['mrss90'] = orderLog['mrss90'].astype(float)
    except:
        print(orderLog[orderLog['mrss90'] == '-'])
        orderLog = orderLog[orderLog['mrss90'] != '-']
        orderLog['mrss90'] = orderLog['mrss90'].astype(float)
    try:
        orderLog['aaa'] = orderLog['aaa'].astype(float)
    except:
        print(orderLog[orderLog['aaa'] == '-'])
        orderLog = orderLog[orderLog['aaa'] != '-']
        orderLog['aaa'] = orderLog['aaa'].astype(float)
    
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
    orderLog.loc[(orderLog['orderDirection'] >= 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
    orderLog.loc[(orderLog['orderDirection'] < 1) &\
             (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    


    orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
    orderLog['mrstaat'] == 3000, '2. statwo', np.where(
    orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
    print(orderLog[(orderLog['sta'] == 'else') & (orderLog['updateType'] == 0)].groupby(['date', 'accCode'])['secid'].size())
    print(orderLog[(orderLog['sta'] == 'else') & (orderLog['date'] == cur) & (orderLog['updateType'] == 0)])
    m1 = orderLog[(orderLog['sta'] == 'else') & (orderLog['date'] == cur) & (orderLog['updateType'] == 0)].shape[0]
    if m1 != 0:
        count += 1
        body += str(count) + '. There are ' + str(m1) + ' orders with invalid strategy.<div>'                    
    orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
    print(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())
    
 



  
    body += '<p><b>II. fill rate</b></p>'

    import pandas as pd
    def convertToHtml(result,title):
        #将数据转换为html的table
        #result是list[list1,list2]这样的结构
        #title是list结构；和result一一对应。titleList[0]对应resultList[0]这样的一条数据对应html表格中的一列
        d = {}
        index = 0
        for t in title:
            d[t]=result[index]
            index = index+1
        df = pd.DataFrame(d)
        df = df[title]
        h = df.to_html(index=False)
        return h

    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
    print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))
    body += '%.2f%% SZE orders triggered by msg data<div>'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100)
    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
    r1 = placeSZE[placeSZE['isMsg'] == 1].groupby(['date', 'accCode'])['secid'].size().reset_index()
    r2 = placeSZE.groupby(['date', 'accCode'])['secid'].size().reset_index()
    re = pd.merge(r1, r2, on=['date', 'accCode'], how='inner')
    re['perc'] = re['secid_x'] / re['secid_y']
    print(re[re['perc'] < 0.8])
    for i in re[re['perc'] < 0.8]['accCode'].unique():
        a = list(re[(re['perc'] < 0.8) & (re['accCode'] == i)]['date'].unique())
        a = ', '.join([str(x) for x in a])
        body += 'accCode ' + str(i) + ' SZE orders on ' + a + ' has msg triggered percentage < 80%<div>'

    orderLog['tag'] = 'previous'
    orderLog.loc[orderLog['date'] == cur, 'tag'] = 'current'
    o1 = orderLog[orderLog['updateType'] == 0].groupby(['tag', 'exchange'])['orderNtl'].sum().reset_index()
    o2 = orderLog[orderLog['updateType'] == 4].groupby(['tag', 'exchange'])['tradeNtl'].sum().reset_index()
    o = pd.merge(o1, o2, on=['tag', 'exchange'])
    o['perc'] = o['tradeNtl'] / o['orderNtl']
    o['perc'] = o['perc'].apply(lambda x: '%.f'%(x*100))
    o1 = o[o['tag'] == 'current']
    o1 = o1.rename(columns={'perc':'cur_perc'})
    o2 = o[o['tag'] == 'previous']
    o2 = o2.rename(columns={'perc':'prev_perc'})
    re = pd.merge(o1[['exchange', 'cur_perc']], o2[['exchange', 'prev_perc']], on='exchange')
    from IPython.display import display, HTML
    display(HTML(re.groupby(['exchange']).first().to_html()))
    result = [re['exchange'].values, re['cur_perc'].values, re['prev_perc'].values]
    title=['exchange', 'cur_perc', 'prev_perc']
    body += convertToHtml(result,title)

    orderLog['num'] = orderLog.groupby(['exchange', 'colo', 'accCode', 'secid'])['tag'].transform('nunique')
    checkLog = orderLog[orderLog['num'] == 2]
    # checkLog = orderLog
    d1 = checkLog[(checkLog['updateType'] == 0)].groupby(['tag', 'exchange', 'sta', 'colo', 'accCode'])['orderNtl'].sum().reset_index()
    d2 = checkLog[(checkLog['updateType'] == 4)].groupby(['tag', 'exchange', 'sta', 'colo', 'accCode'])['tradeNtl'].sum().reset_index()
    dd = pd.merge(d1, d2, on=['tag', 'exchange', 'sta', 'colo', 'accCode'])
    dd['fill rate'] = dd['tradeNtl'] / dd['orderNtl']
    add = checkLog[(checkLog['updateType'] == 0)].groupby(['tag', 'exchange', 'sta', 'colo', 'accCode'])['order'].nunique().reset_index()
    add = add.rename(columns={'order':'num'})
    dd = pd.merge(dd, add, on=['tag', 'exchange', 'sta', 'colo', 'accCode'])
    prev = dd[dd['tag'] == 'previous']
    prev = prev.rename(columns={'num':'prev_num', 'fill rate':'prev_fillRate'})
    cur = dd[dd['tag'] == 'current']
    cur = cur.rename(columns={'num':'cur_num', 'fill rate':'cur_fillRate'})
    report = pd.merge(prev[['exchange', 'sta', 'colo', 'accCode', 'prev_num', 'prev_fillRate']], 
                  cur[['exchange', 'sta', 'colo', 'accCode', 'cur_num', 'cur_fillRate']], on=['exchange', 'sta', 'colo', 'accCode'], how='inner')

    a1 = checkLog[(checkLog['updateType'] == 0) & (checkLog['tag'] == 'previous')].groupby(['date', 'exchange', 'sta', 'colo', 'accCode'])['orderNtl'].sum().reset_index()
    a2 = checkLog[(checkLog['updateType'] == 4) & (checkLog['tag'] == 'previous')].groupby(['date', 'exchange', 'sta', 'colo', 'accCode'])['tradeNtl'].sum().reset_index()
    aa = pd.merge(a1, a2, on=['date', 'exchange', 'sta', 'colo', 'accCode'])
    aa['fill rate'] = aa['tradeNtl'] / aa['orderNtl']
    aa['count'] = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['date'].transform('nunique')
    aa1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['fill rate'].describe()[['mean', 'std']].reset_index()
    aa1 = aa1.fillna(0)
    aa = pd.merge(aa, aa1, on=['exchange', 'sta', 'colo', 'accCode'])
    for j in [1]:
        aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
        aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
        aa['count1'] = np.where((aa['fill rate'] <= aa[str(j) + 'std_high']) & (aa['fill rate'] >= aa[str(j) + 'std_low']), 1, 0)
        re1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count1'].sum().reset_index()
        re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count'].first().reset_index()
        re1 = pd.merge(re2, re1, on=['exchange', 'sta', 'colo', 'accCode'])
        re1['count1'] = re1['count1'] / re1['count']
        re1 = re1.rename(columns={'count1':str(j)+'*std'})
    for j in [1.5, 2, 2.5, 3, 3.5, 4, 4.5]:
        aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
        aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
        aa['count1'] = np.where((aa['fill rate'] <= aa[str(j) + 'std_high']) & (aa['fill rate'] >= aa[str(j) + 'std_low']), 1, 0)
        re = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count1'].sum().reset_index()
        re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode'])['count'].first().reset_index()
        re = pd.merge(re2, re, on=['exchange', 'sta', 'colo', 'accCode'])
        re['count1'] = re['count1'] / re['count']
        re = re.rename(columns={'count1':str(j)+'*std'})
        re1 = pd.merge(re1, re[['exchange', 'sta', 'colo', 'accCode', str(j)+'*std']], on=['exchange', 'sta', 'colo', 'accCode'])
    nc = []
    for i in range(0, re1.shape[0]):
        nc.append(np.float(re1.columns[5:][(re1.iloc[i, 5:] == 1)][0].split('*')[0]))
    re1['n'] = nc
    print(re1.shape[0])
    print(aa1.shape[0])
    aa1 = pd.merge(aa1, re1[['exchange', 'sta', 'colo', 'accCode', 'n']], on=['exchange', 'sta', 'colo', 'accCode'])
    aa1['min'] = aa1['mean'] - aa1['std'] * aa1['n']
    aa1['max'] = aa1['mean'] + aa1['std'] * aa1['n']
    report = pd.merge(report, aa1, on=['exchange', 'sta', 'colo', 'accCode'], how='left')
    assert(report[report['mean'].isnull()].shape[0] == 0)
    print(report[((report['cur_fillRate'] > report['max']) | (report['cur_fillRate'] < report['min'])) & (report['cur_num'] > 100) & (abs(report['cur_fillRate'] - report['prev_fillRate']) > 0.15)].groupby(['exchange', 'sta', 'colo', 'accCode'])['prev_fillRate', 'cur_fillRate'].first())
    report = report[((report['cur_fillRate'] > report['max']) | (report['cur_fillRate'] < report['min'])) & (report['cur_num'] > 100) & (abs(report['cur_fillRate'] - report['prev_fillRate']) > 0.15)].groupby(['exchange', 'sta', 'colo', 'accCode'])['prev_fillRate', 'cur_fillRate'].first().reset_index()

    for cols in ['prev_fillRate', 'cur_fillRate']:
        report[cols] = report[cols].apply(lambda x: '%.f%%'%(100*x))
    from IPython.display import display, HTML
    display(HTML(report.groupby(['exchange', 'sta', 'colo', 'accCode']).first().to_html()))
    body += '<div>In the following cases, fill rate under given accCode pass the hurdle we set:'
    result = [report['exchange'].values, report['sta'].values, report['colo'].values, report['accCode'].values, 
             report['prev_fillRate'].values, report['cur_fillRate'].values]
    title = ['exchange', 'sta', 'colo', 'accCode', 'prev_fillRate', 'cur_fillRate']
    body += convertToHtml(result,title)


    
    
    
    body += '<p><b>III. internal latency</b></p>'

    orderLog['num'] = orderLog.groupby(['exchange', 'colo', 'accCode', 'secid'])['tag'].transform('nunique')
    checkLog = orderLog[(orderLog["updateType"] == 0) & (orderLog['num'] == 2)]
    checkLog = checkLog[checkLog['caamd'] != 0]
    checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
    SZE = checkLog[checkLog['secid'] >= 2000000]
    SSE = checkLog[checkLog['secid'] < 2000000]
    SZE = SZE[SZE['isMsg'] == 1]
    c1 = SZE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].quantile(.95).reset_index()
    c2 = SZE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].median().reset_index()
    c3 = SZE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].count().reset_index()

    re1 = pd.merge(c3, c1, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
    re1 = re1.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
    re1 = pd.merge(re1, c2, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
    re1 = re1.rename(columns = {'internal_latency': 'median'})
    re1['isMsg'] = 1

    c1 = SSE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].quantile(.95).reset_index()
    c2 = SSE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].median().reset_index()
    c3 = SSE.groupby(['tag', "exchange", "colo", "accCode", "sta", "isMsg"])["internal_latency"].count().reset_index()

    re2 = pd.merge(c3, c1, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
    re2 = re2.rename(columns = {'internal_latency_x': 'count', 'internal_latency_y': '95 percentile'})
    re2 = pd.merge(re2, c2, on=['tag', "exchange", "colo", "accCode", "sta", "isMsg"])
    re2 = re2.rename(columns = {'internal_latency': 'median'})
    re2

    re = pd.concat([re1, re2]).reset_index(drop=True)

    for col in ['isMsg','median', '95 percentile']:
        re[col] = re[col].astype(int)

    re1 = re[re['tag'] == 'current']
    re1 = re1.rename(columns={'count':'cur_count', 'median':'cur_med', '95 percentile':'cur_95p'})
    re2 = re[re['tag'] == 'previous']
    re2 = re2.rename(columns={'count':'prev_count', 'median':'prev_med', '95 percentile':'prev_95p'})
    report = pd.merge(re1, re2, on=['exchange', 'colo', 'accCode', 'sta', 'isMsg'])

    checkLog[(checkLog['exchange'] == 'SSE') | ((checkLog['exchange'] == 'SZE') & (checkLog['isMsg'] == 1))]
    aa = checkLog[(checkLog['tag'] == 'previous')].groupby(['date', 'exchange', 'sta', 'colo', 'accCode', 'isMsg'])['internal_latency'].median().reset_index()
    aa['count'] = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['date'].transform('nunique')
    aa1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['internal_latency'].describe()[['mean', 'std']].reset_index()
    aa1 = aa1.fillna(0)
    aa = pd.merge(aa, aa1, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
    for j in [1]:
        aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
        aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
        aa['count1'] = np.where((aa['internal_latency'] <= aa[str(j) + 'std_high']) & (aa['internal_latency'] >= aa[str(j) + 'std_low']), 1, 0)
        re1 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count1'].sum().reset_index()
        re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count'].first().reset_index()
        re1 = pd.merge(re2, re1, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
        re1['count1'] = re1['count1'] / re1['count']
        re1 = re1.rename(columns={'count1':str(j)+'*std'})
    for j in [1.5, 2, 2.5, 3, 3.5, 4, 4.5]:
        aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
        aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
        aa['count1'] = np.where((aa['internal_latency'] <= aa[str(j) + 'std_high']) & (aa['internal_latency'] >= aa[str(j) + 'std_low']), 1, 0)
        re = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count1'].sum().reset_index()
        re2 = aa.groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['count'].first().reset_index()
        re = pd.merge(re2, re, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
        re['count1'] = re['count1'] / re['count']
        re = re.rename(columns={'count1':str(j)+'*std'})
        re1 = pd.merge(re1, re[['exchange', 'sta', 'colo', 'accCode', 'isMsg', str(j)+'*std']], on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
    nc = []
    for i in range(0, re1.shape[0]):
        nc.append(np.float(re1.columns[6:][(re1.iloc[i, 6:] == 1)][0].split('*')[0]))
    re1['n'] = nc
    print(re1.shape[0])
    print(aa1.shape[0])
    aa1 = pd.merge(aa1, re1[['exchange', 'sta', 'colo', 'accCode', 'isMsg', 'n']], on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'])
    aa1['min'] = aa1['mean'] - aa1['std'] * aa1['n']
    aa1['max'] = aa1['mean'] + aa1['std'] * aa1['n']
    report = pd.merge(report, aa1, on=['exchange', 'sta', 'colo', 'accCode', 'isMsg'], how='left')
    assert(report[report['mean'].isnull()].shape[0] == 0)

    from IPython.display import display, HTML
    display(HTML(report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                        & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 20)] \
                 .groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['prev_med', 'cur_med', 'prev_95p', 'cur_95p'].first().to_html()))
    report = report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                        & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 20)] \
                 .groupby(['exchange', 'sta', 'colo', 'accCode', 'isMsg'])['prev_med', 'cur_med', 'prev_95p', 'cur_95p'].first().reset_index()

    body += '<div>In the following cases, internal latency under given accCode pass the hurdle we set:'
    result = [report['exchange'].values, report['sta'].values, report['colo'].values, report['accCode'].values, report['isMsg'].values, 
             report['prev_med'].values, report['prev_95p'].values, report['cur_med'].values, report['cur_95p'].values]
    title = ['exchange', 'sta', 'colo', 'accCode', 'isMsg', 'prev_med', 'prev_95p', 'cur_med', 'cur_95p']
    body += convertToHtml(result,title)
 




    body += '<p><b>IV. tickToMBD</b></p>'

    import os
    import glob
    import datetime
    import numpy as np
    import pandas as pd

    checkLog = orderLog[~orderLog['start_time'].isnull()]
    checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
    checkLog = checkLog[~checkLog['accCode'].isnull()]
    checkLog['tag'] = 'previous'
    checkLog.loc[checkLog['date'] == orderLog['date'].max(), 'tag'] = 'current'

    cc1 = checkLog
    cc1 = cc1.reset_index(drop=True)
    cc1['ordering'] = cc1.index
    cc1['time_diff'] = cc1['caa_orderLog'] - cc1['start_time']
    cc1['colo1'] = cc1['colo'].str[:2] + cc1['colo'].str[3:5] + cc1['colo'].str[6:8]
    cc1['colo_broker'] = cc1['colo1'] + '_' + cc1["accCode"].astype(int).astype(str)


    re1 = cc1.groupby(['tag', 'sta', 'colo', 'accCode'])['time_diff'].describe().fillna(0).astype(int).reset_index()
    # re1 = re1[re1['count'] > 20].reset_index()
    c1 = cc1.groupby(['tag', 'sta', 'colo', 'accCode'])['time_diff'].apply(lambda x: x.describe([0.1])['10%']).astype(int).reset_index()
    c1 = c1.rename(columns={"time_diff":"10%"})
    re1 = pd.merge(re1, c1[['tag', 'sta', 'colo', 'accCode', '10%']], on=['tag', 'sta', 'colo', 'accCode'])
    c1 = cc1.groupby(['tag', 'sta', 'colo', 'accCode'])['time_diff'].apply(lambda x: x.describe([0.9])['90%']).astype(int).reset_index()
    c1 = c1.rename(columns={"time_diff":"90%"})
    re1 = pd.merge(re1, c1[['tag', 'sta', 'colo', 'accCode', '90%']], on=['tag', 'sta', 'colo', 'accCode'])
    ree1 = re1[re1['tag'] == 'previous']
    ree1 = ree1.rename(columns={'50%':'prev_med', 'count':'prev_count'})
    ree2 = re1[re1['tag'] == 'current']
    ree2 = ree2.rename(columns={'50%':'cur_med', 'count':'cur_count'})
    report = pd.merge(ree1[['sta', 'colo', 'accCode', 'prev_count', 'prev_med']], 
                   ree2[['sta', 'colo', 'accCode', 'cur_count', 'cur_med']], on=['sta', 'colo', 'accCode'])


    aa = cc1[(cc1['updateType'] == 0) & (cc1['tag'] == 'previous')].groupby(['date', 'sta', 'colo', 'accCode'])['time_diff'].median().reset_index()
    aa['count'] = aa.groupby(['sta', 'colo', 'accCode'])['date'].transform('nunique')
    aa1 = aa.groupby(['sta', 'colo', 'accCode'])['time_diff'].describe()[['mean', 'std']].reset_index()
    aa1 = aa1.fillna(0)
    aa = pd.merge(aa, aa1, on=['sta', 'colo', 'accCode'])
    for j in [1]:
        aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
        aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
        aa['count1'] = np.where((aa['time_diff'] <= aa[str(j) + 'std_high']) & (aa['time_diff'] >= aa[str(j) + 'std_low']), 1, 0)
        re1 = aa.groupby(['sta', 'colo', 'accCode'])['count1'].sum().reset_index()
        re2 = aa.groupby(['sta', 'colo', 'accCode'])['count'].first().reset_index()
        re1 = pd.merge(re2, re1, on=['sta', 'colo', 'accCode'])
        re1['count1'] = re1['count1'] / re1['count']
        re1 = re1.rename(columns={'count1':str(j)+'*std'})
    for j in [1.5, 2, 2.5, 3, 3.5, 4, 4.5]:
        aa[str(j) + 'std_low'] = aa['mean'] - j*aa['std']
        aa[str(j) + 'std_high'] = aa['mean'] + j*aa['std']
        aa['count1'] = np.where((aa['time_diff'] <= aa[str(j) + 'std_high']) & (aa['time_diff'] >= aa[str(j) + 'std_low']), 1, 0)
        re = aa.groupby(['sta', 'colo', 'accCode'])['count1'].sum().reset_index()
        re2 = aa.groupby(['sta', 'colo', 'accCode'])['count'].first().reset_index()
        re = pd.merge(re2, re, on=['sta', 'colo', 'accCode'])
        re['count1'] = re['count1'] / re['count']
        re = re.rename(columns={'count1':str(j)+'*std'})
        re1 = pd.merge(re1, re[['sta', 'colo', 'accCode', str(j)+'*std']], on=['sta', 'colo', 'accCode'])
    nc = []
    for i in range(0, re1.shape[0]):
        nc.append(np.float(re1.columns[6:][(re1.iloc[i, 6:] == 1)][0].split('*')[0]))
    re1['n'] = nc
    print(re1.shape[0])
    print(aa1.shape[0])
    aa1 = pd.merge(aa1, re1[['sta', 'colo', 'accCode', 'n']], on=['sta', 'colo', 'accCode'])
    aa1['min'] = aa1['mean'] - aa1['std'] * aa1['n']
    aa1['max'] = aa1['mean'] + aa1['std'] * aa1['n']
    report = pd.merge(report, aa1, on=['sta', 'colo', 'accCode'], how='left')
    assert(report[report['mean'].isnull()].shape[0] == 0)

    from IPython.display import display, HTML
    display(HTML(report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                        & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 1000)] \
                 .groupby(['sta', 'colo', 'accCode'])['prev_med', 'cur_med'].first().to_html()))
    report = report[((report['cur_med'] > report['max']) | (report['cur_med'] < report['min'])) \
                        & (report['cur_count'] > 100) & (abs(report['cur_med'] - report['prev_med']) > 1000)] \
                 .groupby(['sta', 'colo', 'accCode'])['prev_med', 'cur_med'].first().reset_index()
    body += '<div>In the following cases, tickToMBD under given accCode pass the hurdle we set:<div>'
    result = [report['sta'].values, report['colo'].values, report['accCode'].values,
             report['prev_med'].values, report['cur_med'].values]
    title = ['sta', 'colo', 'accCode', 'prev_med', 'cur_med']
    body += convertToHtml(result,title)

    import os
    import glob
    import datetime
    import numpy as np
    import pandas as pd

    
    
    
    
    
    body += '<p><b>V. order return</b></p>'
    checkLog = orderLog[orderLog['updateType'] == 4]
    if checkLog[checkLog['absFilledThisUpdate'] == 0].shape[0] != 0:
        print('There are stocks with zero trade size')
        print(checkLog[checkLog['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
        checkLog = checkLog[checkLog['absFilledThisUpdate'] != 0]
    if checkLog[checkLog['beta_60'].isnull()].shape[0] != 0:
        print('There are stocks with null beta')
        print(checkLog[checkLog['beta_60'].isnull()])
        checkLog = checkLog[~checkLog['beta_60'].isnull()]
    checkLog['max_trade'] = checkLog.groupby('order')['absOrderSizeCumFilled'].transform('max')
    checkLog['last'] = 0
    checkLog.loc[checkLog['max_trade'] == checkLog['absOrderSizeCumFilled'], 'last'] = 1
    checkLog["buyRet"] = np.where(checkLog["orderDirection"].isin([1, 2]), checkLog["adjMid_F90s"] / checkLog["tradePrice"] - 1, np.nan)
    checkLog["buyRet1"] = np.where(checkLog["orderDirection"].isin([1, 2]), checkLog["adjMid_F300s"] / checkLog["tradePrice"] - 1, np.nan)
    checkLog["sellRet"] = np.where(checkLog["orderDirection"].isin([-1, -2]), checkLog["tradePrice"] / checkLog["adjMid_F90s"] - 1, np.nan)
    checkLog["sellRet1"] = np.where(checkLog["orderDirection"].isin([-1, -2]), checkLog["tradePrice"] / checkLog["adjMid_F300s"] - 1, np.nan)
    checkLog["buyNum"] = np.where((checkLog["orderDirection"].isin([1, 2])) & (checkLog['last'] == 1), 1, 0)
    checkLog["sellNum"] = np.where((checkLog["orderDirection"].isin([-1, -2])) & (checkLog['last'] == 1), 1, 0)
    checkLog["server"] = checkLog["colo"].apply(lambda x: x.split("_")[0] + x.split("_")[1] + x.split("_")[2])
    checkLog["server_account"] = checkLog["server"] + '_' + checkLog['accCode'].astype('str')
    df = checkLog[(checkLog['ars']%10 == 1)]

    df['tradeNtl'] = df['tradePrice']*df['absFilledThisUpdate']
    df["buyNtl"] = np.where(~df["buyRet"].isnull(), df["tradeNtl"], np.nan)
    df["sellNtl"] = np.where(~df["sellRet"].isnull(), df["tradeNtl"], np.nan)
    df["sumbuyNtl"] = df.groupby(["date", "exchange", "sta", "server_account"])["buyNtl"].transform(sum)
    df["sumsellNtl"] = df.groupby(["date", "exchange", "sta", "server_account"])["sellNtl"].transform(sum)

    df["sumsellRet"] = df["tradeNtl"] * df["sellRet"]
    df["sumsellRet"] = df.groupby(["date", "exchange", "sta", "server_account"])["sumsellRet"].transform(sum)


    df["sumbuyRet"] = df["tradeNtl"] * df["buyRet"]
    df["sumbuyRet"] = df.groupby(["date", "exchange", "sta", "server_account"])["sumbuyRet"].transform(sum)

    df["buyRet"] = df["sumbuyRet"] / df["sumbuyNtl"]
    df["sellRet"] = df["sumsellRet"] / df["sumsellNtl"]
    df["buyOrderNum"] = df.groupby(["date", "exchange", "sta", "server_account"])["buyNum"].transform(sum)
    df["sellOrderNum"] = df.groupby(["date", "exchange", "sta", "server_account"])["sellNum"].transform(sum)


    from IPython.display import display, HTML
    for col in ["buyRet", "sellRet"]:
        df[col] = (df[col] * 10000).round(2)

    re = df.groupby(["exchange", "date", "sta", "server_account"])["buyOrderNum", "buyRet", "sellOrderNum", "sellRet"].first().reset_index()
    re1 = re[(re['sellRet'] < -20) & (re['sellOrderNum'] > 100)]
    re2 = re[(re['buyRet'] < -20) & (re['buyOrderNum'] > 100)]

    display(HTML(re1.groupby(["exchange", 'date', 'sta', "server_account"])[['sellOrderNum', 'sellRet']].first().to_html()))
    display(HTML(re2.groupby(["exchange", 'date', 'sta', "server_account"])[['buyOrderNum', 'buyRet']].first().to_html()))
    pr1 = re1[re1['date'] == checkLog['date'].max()].groupby(["exchange", 'date', 'sta', "server_account"])[['sellOrderNum', 'sellRet']].first().reset_index()
    pr2 = re2[re2['date'] == checkLog['date'].max()].groupby(["exchange", 'date', 'sta', "server_account"])[['buyOrderNum', 'buyRet']].first().reset_index()

    if pr1.shape[0] != 0:
        body += '<div>In the following cases, sell order return under given strategy and accCode < -20bps:<div>'
        result = [pr1['exchange'].values, pr1['date'].values, pr1['sta'].values,
                 pr1['server_account'].values, pr1['sellOrderNum'].values, pr1['sellRet'].values]
        title = ['exchange', 'date', 'sta', 'server_account', 'sellOrderNum', 'sellRet']
        body += convertToHtml(result,title)

    if pr2.shape[0] != 0:
        body += '<div>In the following cases, buy order return under given strategy and accCode < -20bps:<div>'
        result = [pr2['exchange'].values, pr2['date'].values, pr2['sta'].values,
                 pr2['server_account'].values, pr2['buyOrderNum'].values, pr2['buyRet'].values]
        title = ['exchange', 'date', 'sta', 'server_account', 'buyOrderNum', 'buyRet']
        body += convertToHtml(result,title)

    pr1 = re1.groupby(['exchange', 'sta', 'server_account'])['date'].size()[
        re1.groupby(['exchange', 'sta', 'server_account'])['date'].size() >= 2].reset_index().sort_values(by='date', ascending=False)
    pr2 = re2.groupby(['exchange', 'sta', 'server_account'])['date'].size()[
        re2.groupby(['exchange', 'sta', 'server_account'])['date'].size() >= 2].reset_index().sort_values(by='date', ascending=False)
    add1 = pd.merge(re, pr1, on=['exchange', 'sta', 'server_account'], how='inner')
    add1 = add1[(add1['sellRet'] < -20) & (add1['sellOrderNum'] > 100)]
    add2 = pd.merge(re, pr2, on=['exchange', 'sta', 'server_account'], how='inner')
    add2 = add2[(add2['buyRet'] < -20) & (add2['buyOrderNum'] > 100)]
    
    if add1.shape[0] != 0:
        add1 = add1.groupby(['exchange', 'sta', 'server_account'])['sellRet'].describe()[['count', 'mean', 'min', 'max']].reset_index().sort_values(by='count', ascending=False)
        for i in ['mean', 'min', 'max']:
            add1[i] = add1[i].round(2)
        for i in ['count']:
            add1[i] = add1[i].astype(int)
        body += '<div>In the following cases, more than 2 days sell order return under given strategy and accCode < -20bps in previous 10 days (include today):<div>'
        result = [add1['exchange'].values, add1['sta'].values,
                 add1['server_account'].values, add1['count'].values, add1['mean'].values, add1['min'].values, add1['max'].values]
        title = ['exchange', 'sta', 'server_account', 'count', 'mean', 'min', 'max']
        body += convertToHtml(result,title)

    if add2.shape[0] != 0:
        add2 = add2.groupby(['exchange', 'sta', 'server_account'])['buyRet'].describe()[['count', 'mean', 'min', 'max']].reset_index().sort_values(by='count', ascending=False)
        for i in ['mean', 'min', 'max']:
            add2[i] = add2[i].round(2)
        for i in ['count']:
            add2[i] = add2[i].astype(int)
        body += '<div>In the following cases, more than 2 days buy order return under given strategy and accCode < -20bps in previous 10 days (include today):<div>'
        result = [add2['exchange'].values, add2['sta'].values,
                 add2['server_account'].values, add2['count'].values, add2['mean'].values, add2['min'].values, add2['max'].values]
        title = ['exchange', 'sta', 'server_account', 'count', 'mean', 'min', 'max']
        body += convertToHtml(result,title)
    
    
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart    
    
    title = str(orderLog['date'].max()) + ' daily report'
    body = body + "</div></body></html>"
    smtp_server = '42.120.226.4' # 'smtp.mxhichina.com'
    user = 'zhenyu.yin@general-int.com'
    passwd = 'Yqzy0063!'
    from_addr = 'zhenyu.yin@general-int.com'
    to_addr = ['zhenyu.yin@general-int.com', 'kevin.zhang@general-int.com']
#     to_addr = ['zhenyu.yin@general-int.com']

    msg = MIMEMultipart()
    msg['From'] = from_addr
    msg['To'] = ', '.join(to_addr)
    msg['Subject'] = title
    txt = MIMEText(body, _subtype='html', _charset='UTF-8')
    msg.attach(txt)

    smtp = None
    while True:
        try:
            smtp = smtplib.SMTP(smtp_server)
            print('smtp server connected')
            smtp.login(user, passwd)
            print('login')
            break
        except Exception as e:
            print(e)
    print('send mail')
    smtp.sendmail(from_addr, to_addr, msg.as_string())
    smtp.quit()

    
    
    
    
from twisted.internet import task, reactor
import schedule


def sleeptime(hour,min,sec):
    return hour*3600 + min*60 + sec


class Test(object):
    def __init__(self):
        self.status = True        
    def test(self):
        while self.status == True:
            try:
                print(datetime.datetime.now())
#                 date = (datetime.datetime.today()).strftime('%Y%m%d')
                date = '20210113'
                readPath = '/mnt/orderLog/data/***'
                dataPathLs = np.array(glob.glob(readPath))
                dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
                assert(np.max(dateLs) == int(date))
                print('We start to generate report now')
                generate_report(date)
                self.status = False
            except Exception as e:
                print(e)
                print('still wait for data coming')  
                second = sleeptime(0,5,0)
                time.sleep(second)

test1 = Test()
schedule.every().day.at("22:03").do(test1.test)
while True:
    schedule.run_pending()
    time.sleep(1)

2021-01-13 22:03:00.388050

still wait for data coming
2021-01-13 22:08:00.495325
We start to generate report now
start
-----------------------------------------------------------------------------------------------
load data
/mnt/orderLog/data/20201229.pkl
/mnt/orderLog/data/20201230.pkl
/mnt/orderLog/data/20201231.pkl
/mnt/orderLog/data/20210104.pkl
/mnt/orderLog/data/20210105.pkl
/mnt/orderLog/data/20210106.pkl
/mnt/orderLog/data/20210107.pkl
/mnt/orderLog/data/20210108.pkl
/mnt/orderLog/data/20210111.pkl
/mnt/orderLog/data/20210112.pkl
/mnt/orderLog/data/20210113.pkl
There are ticks with orderDirection 0
             date      colo  accCode    secid  vai  updateType      sdd  \
48658    20201229  zt_96_01   966301  1600376   -1           1     -1.0   
48659    20201229  zt_96_01   966301  1600376   -1           1     -1.0   
48660    20201229  zt_96_01   966301  1600376   -1           1     -1.0   
247510   20201229  zt_52_04   537301  1603786   -1           1     -1.0   
271265   

/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
     secid  accCode      colo      vai  updateType         sdd  internalId  \
0  2300271     8856  zs_88_04  22400.0           0  93007340.0        30.0   
1  2300271     8856  zs_88_04  22400.0           2        -1.0        30.0   
2  2300271     8856  zs_88_04  22400.0           1     34209.0        30.0   
3  2300271     8856  zs_88_04  22400.0           3        -1.0        30.0   
4  2300271     8856  zs_88_04  22400.0           0  93009860.0        34.0   
5  2300271     8856  zs_88_04  22400.0           2        -1.0        34.0   
6  2300271     8856  zs_88_04  22400.0           2        -1.0        34.0   

   orderId  absOrderSize  absFilledThisUpdate  absOrderSizeCumFilled  \
0     30.0           100                    0                      0   
1     30.0           100                    0                      0   
2     30.0           100                    0                

/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


opposite direction for same date, same secid, same vai
accCode
5222      1
5225      1
5230      2
5232      1
5289      1
5290      3
5291      2
5326      2
5329      3
5377      1
5384      2
5386      1
5470      2
5474      2
6480      3
6623      2
6627      4
6631      1
6634      2
6878      1
8924      1
8943      2
8970      1
9208      1
9243      1
9248      1
9448      5
9454      1
9461      1
9471      1
9551      1
9655      2
9741      1
9754      2
9756      1
9765      2
523101    1
527101    3
528701    2
529001    1
529101    1
897102    2
974102    1
Name: orderDirection, dtype: int64
2. same date, secid, vai, accCode: one insertion
more than one insertion at same time
             date      colo  accCode    secid        vai          sdd  \
1574263  20210108  zs_96_08     6237  1603317  1375708.0  103134000.0   

         clockAtArrival  
1574263               2  
date
20210108    1
Name: accCode, dtype: int64
date      accCode
20210108  6237       1
Name: sdd, dt

...

,,,prev_med,cur_med
sta,colo,accCode,,


/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:773: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


There are stocks with zero trade size
colo      accCode
zs_88_04  8924       6
zt_88_06  8886       2
          8971       1
Name: secid, dtype: int64
There are stocks with null beta
         Unnamed: 0  ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
24029       24058.0         0.0  0.001064                 1000          1700   
24034       24062.0         0.0  0.000567                 2400          2400   
24037       24064.0         0.0  0.003055                 2000          2000   
24040       24066.0         0.0  0.003466                  800           800   
51577       51637.0         0.0 -0.000262                  400           400   
...             ...         ...       ...                  ...           ...   
2341802    879156.0         0.0  0.002240                  100           100   
2341805    879035.0         0.0  0.001978                  400           400   
2341808    879037.0         0.0  0.003710                  300           300   
2341819    879039

/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:814: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:815: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:816: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

smtp server connected
login
send mail
